> This is one of the 100 recipes of the [IPython Cookbook](http://ipython-books.github.io/), the definitive guide to high-performance scientific computing and data science in Python.


# 14.2. Analyzing a social network with NetworkX

First, you need the *twitter* Python package for this recipe. You can install it with `pip install twitter`. (https://pypi.python.org/pypi/twitter)

Then, you need to obtain authentication codes in order to access Twitter data. The procedure is free. In addition to a Twitter account, you also need to create an *Application* on the Twitter Developers website. Then, you will be able to retrieve the OAuth authentication codes that are required for this recipe. (https://dev.twitter.com/apps)

**Note that access to the Twitter API is not unlimited**. Most methods can only be called a few times within a given time window. Unless you study small networks or look at small portions of large networks, you will need to throttle your requests. In this recipe, we only consider a small portion of the network, so that the API limit should not be reached. Otherwise, you will have to wait a few minutes before the next time window starts. (https://dev.twitter.com/docs/rate-limiting/1.1/limits)

1. Let's import a few packages.

In [ ]:
import math
import json
import twitter
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

2.  You need to create a `twitter.txt` text file in the current folder with the four authentication keys. You will find those in your Twitter Developers Application page, *OAuth tool* section. (https://dev.twitter.com/apps)

In [ ]:
(CONSUMER_KEY, CONSUMER_SECRET, 
OAUTH_TOKEN, OAUTH_TOKEN_SECRET) = open('twitter.txt', 'r').read().splitlines()

3. We now create the `Twitter` instance that will give us access to the Twitter API.

In [ ]:
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
tw = twitter.Twitter(auth=auth)

4. We use the latest 1.1 version of the Twitter API in this recipe. The `twitter` library defines a direct mapping between the REST API and the attributes of the `Twitter` instance. Here, we execute the `account/verify_credentials` REST request to obtain the user id of the authenticated user (me here, or you if you execute this notebook yourself!).

In [ ]:
me = tw.account.verify_credentials()

In [ ]:
myid = me['id']

5. Let's define a simple function that returns the identifiers of all followers of a given user (the authenticated user by default).

In [ ]:
def get_followers_ids(uid=None):
    # Retrieve the list of followers' ids of the specified user.
    return tw.followers.ids(user_id=uid)['ids']

In [ ]:
# We get the list of my followers.
my_followers_ids = get_followers_ids()

6. Now, we define a function that retrieves the full profile of Twitter users. Since the `users/lookup` batch request is limited to 100 users per call, and that only a small number of calls are allowed within a time window, we only look at a subset of all the followers.

In [ ]:
def get_users_info(users_ids, max=500):
    n = min(max, len(users_ids))
    # Get information about those users, using batch requests.
    users = [tw.users.lookup(user_id=users_ids[100*i:100*(i+1)])
             for i in range(int(math.ceil(n/100.)))]
    # We flatten this list of lists.
    users = [item for sublist in users for item in sublist]
    return {user['id']: user for user in users}

In [ ]:
users_info = get_users_info(my_followers_ids)

In [ ]:
# Let's save this dictionary on the disk.
with open('my_followers.json', 'w') as f:
    json.dump(users_info, f, indent=1)

7. We also start to define the graph with the followers, as an adjacency list (technically, a dictionary of lists). This is called the **ego graph**. This graph represents all *following* connections between my followers.

In [ ]:
adjacency = {myid: my_followers_ids}

8. Now, we are going to take a look at the part of the ego graph related to Python. Specifically, we will consider the followers of the 10 most followed users whom descriptions contain "Python".

In [ ]:
my_followers_python = [user for user in users_info.values()
                       if 'python' in user['description'].lower()]

In [ ]:
my_followers_python_best = sorted(my_followers_python, 
        key=lambda u: u['followers_count'])[::-1][:10]

The request for retrieving the followers of a given user is rate-limited. Let's check how many calls remaining we have.

In [ ]:
tw.application.rate_limit_status(resources='followers') \
             ['resources']['followers']['/followers/ids']

In [ ]:
for user in my_followers_python_best:
    # The call to get_followers_ids is rate-limited.
    adjacency[user['id']] = list(set(get_followers_ids(
        user['id'])).intersection(my_followers_ids))

9. Now that our graph is defined as an adjacency list in a dictionary, we will load it in NetworkX.

In [ ]:
g = nx.Graph(adjacency)

In [ ]:
# We only restrict the graph to the users for which we 
# were able to retrieve the profile.
g = g.subgraph(users_info.keys())

In [ ]:
# We also save this graph on disk.
with open('my_graph.json', 'w') as f:
    json.dump(nx.to_dict_of_lists(g), f, indent=1)

In [ ]:
# We remove isolated nodes for simplicity.
g.remove_nodes_from([k for k, d in g.degree().items()
                     if d <= 1])

In [ ]:
# Since I am connected to all nodes, by definition,
# we can remove me for simplicity.
g.remove_nodes_from([myid])

10. Let's take a look at the graph's statistics.

In [ ]:
len(g.nodes()), len(g.edges())

11. We are now going to plot this graph. We will use different sizes and colors for the nodes, according to the number of followers and the number of tweets for each user. Most followed users will be bigger. Most active users will be redder.

In [ ]:
# Update the dictionary.
deg = g.degree()
for user in users_info.values():
    fc = user['followers_count']
    sc = user['statuses_count']
    # Is this user a Pythonista?
    user['python'] = 'python' in user['description'].lower()
    # We compute the node size as a function of the 
    # number of followers.
    user['node_size'] = math.sqrt(1 + 10 * fc)
    # The color is function of its activity on Twitter.
    user['node_color'] = 10 * math.sqrt(1 + sc)
    # We only display the name of the most followed users.
    user['label'] = user['screen_name'] if fc > 2000 else ''

12. Finally, we use the `draw` function to display the graph. We need to specify the node sizes and colors as lists, and the labels as a dictionary.

In [ ]:
node_size = [users_info[uid]['node_size'] for uid in g.nodes()]

In [ ]:
node_color = [users_info[uid]['node_color'] for uid in g.nodes()]

In [ ]:
labels = {uid: users_info[uid]['label'] for uid in g.nodes()}

In [ ]:
plt.figure(figsize=(6,6));
nx.draw(g, cmap=plt.cm.OrRd, alpha=.8,
        node_size=node_size, node_color=node_color,
        labels=labels, font_size=4, width=.1);

## See also

* Manipulate and visualize graphs with NetworkX

> You'll find all the explanations, figures, references, and much more in the book (to be released later this summer).

> [IPython Cookbook](http://ipython-books.github.io/), by [Cyrille Rossant](http://cyrille.rossant.net), Packt Publishing, 2014 (500 pages).